## Install dependencies and setting up GPU enviornment

In [2]:
#!pip install tensorflow==2.0.0-beta1 
#!pip install tqdm

In [3]:
!wget --no-check-certificate \
  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
  -O ./cats_and_dogs_filtered.zip

--2019-08-05 20:56:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.31.144, 2404:6800:4004:800::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.31.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  69.2MB/s    in 0.9s    

2019-08-05 20:56:25 (69.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Dataset Preprocessing

### Import project dependenceies

In [4]:
import os
import zipfile 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0-beta1'

### Unzipping the Dogs vs Cats dataset
  

In [0]:
dataset_path = './cats_and_dogs_filtered.zip'

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode='r')

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Setting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [16]:
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top=False, weights = "imagenet")

9412608/9406464 [==============================] - 2s 0us/step


In [18]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [0]:
base_model.trainable = False

### Defining the custom head for our network

In [20]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [22]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units= 1, activation = 'sigmoid')(global_average_layer)

### Defining the model

In [0]:
model = tf.keras.models.Model(inputs=base_model.input,outputs=prediction_layer)

In [27]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compiling the model

In [0]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy",metrics=["accuracy"])

### Creating Data Generators
Resizing images
    
    Big pre-trained architecture support only certain input sizes.
    
for example: MobileNet (architecture that we use) supports (96,96),(128,128),160,160),(192,192),(224,224).

In [0]:
data_gen_train = ImageDataGenerator(rescale = 1/255.)
data_gen_valid = ImageDataGenerator(rescale = 1/255.)

In [29]:
train_generator =data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128,class_mode ="binary")

Found 2000 images belonging to 2 classes.


In [30]:
valid_generator =data_gen_train.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128,class_mode ="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [32]:
model.fit_generator(train_generator, epochs = 5,validation_data=valid_generator)

Epoch 1/5


RuntimeError: ignored

### Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(vaild_generator)

In [0]:
print(f"accuracy after Transfer Learning: {valid_accuracy*100}%")

## Fine Tuning

there are a few pointers: 
  * DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. the goal of fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
  * Start with the fine tuning AFTER you have finished with the transfer learning step. If we try to perfom Fine tuning immedieately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [0]:
base_model.trainable = True

In [0]:
print(f"number of layers in base model : {len(base_model.layers)}")

In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
             loss='binary_crossentropy',
             metrics=['accuracy'])

### Fine Tuning

In [0]:
model.fit_generator(train_generator,
                   epochs = 5,
                   validation_data=valid_generator)

### Evaluating the fine tuned model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_genrator)

In [0]:
print(f"Validation accuracy after fine tuning : {valid_accuracy*100}%")